<a href="https://colab.research.google.com/github/NAGPALADITI14/Articles_analysis/blob/main/Articles_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 20211030TestAssignment-20241231T052758Z-001.zip to 20211030TestAssignment-20241231T052758Z-001.zip


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In order to unzip the folder kindly remove the spaces from the foldername

In [ ]:
!unzip /content/20211030TestAssignment-20241231T052758Z-001.zip

Archive:  /content/20211030TestAssignment-20241231T052758Z-001.zip
  inflating: 20211030 Test Assignment/MasterDictionary/negative-words.txt  
  inflating: 20211030 Test Assignment/StopWords/StopWords_DatesandNumbers.txt  
  inflating: 20211030 Test Assignment/Output Data Structure.xlsx  
  inflating: 20211030 Test Assignment/MasterDictionary/positive-words.txt  
  inflating: 20211030 Test Assignment/StopWords/StopWords_Geographic.txt  
  inflating: 20211030 Test Assignment/Input.xlsx  
  inflating: 20211030 Test Assignment/StopWords/StopWords_Auditor.txt  
  inflating: 20211030 Test Assignment/StopWords/StopWords_Currencies.txt  
  inflating: 20211030 Test Assignment/StopWords/StopWords_Generic.txt  
  inflating: 20211030 Test Assignment/Objective.docx  
  inflating: 20211030 Test Assignment/Text Analysis.docx  
  inflating: 20211030 Test Assignment/StopWords/StopWords_GenericLong.txt  
  inflating: 20211030 Test Assignment/StopWords/StopWords_Names.txt  


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import re
from nltk.tokenize import sent_tokenize,word_tokenize
import chardet

In [ ]:
pip install textstat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.5 MB/s eta 0:00:00


In [ ]:
from textstat import syllable_count

In [ ]:
def extract_articles(input_file,output_dir):
  df=pd.read_excel(input_file)
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)

  for _, row in df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    try:
      response = requests.get(url)
      soup=BeautifulSoup(response.content,'html.parser')

      title = soup.find('h1',class_='entry-title').get_text(strip=True)
      # print("Title : ",title)

      text_content = ' '.join([div.get_text(strip=True) for div in soup.find_all('div',class_='td-post-content tagdiv-type')])
      with open(f"{output_dir}/{url_id}.txt", "w", encoding="utf-8") as file:
                file.write(title + "\n\n" + text_content)

    except Exception as e:
            print(f"Failed to process URL_ID {url_id}: {e}")

In [ ]:
def load_word_list(file_path):

    with open(file_path, 'rb') as file:
        raw_data = file.read()
        detected = chardet.detect(raw_data)
        encoding = detected.get('encoding', 'utf-8')
        # print(f"Detected encoding for {file_path}: {encoding}")
    with open(file_path, 'r', encoding=encoding , errors= 'replace') as file:
        words = {line.strip().lower() for line in file if line.strip()}
    return words

def load_multiple_word_lists(directory_path):
    combined_words = set()
    for file_name in os.listdir(directory_path):
        file_path = os.path.join(directory_path, file_name)
        if os.path.isfile(file_path) and file_name.endswith('.txt'):
            with open(file_path, 'rb') as f:
                raw_data = f.read()
                detected = chardet.detect(raw_data)
                encoding = detected.get('encoding', 'utf-8')
            try:
              with open(file_path, 'r', encoding='utf-8',errors='replace') as file:
                  combined_words.update({line.strip().lower() for line in file if line.strip()})
            except Exception as e:
                print(f"Error reading file {file_name}: {e}")
    return combined_words

In [ ]:
positive_words = load_word_list('/content/20211030TestAssignment/MasterDictionary/positive-words.txt')
negative_words = load_word_list('/content/20211030TestAssignment/MasterDictionary/negative-words.txt')
stop_words = load_multiple_word_lists('/content/20211030TestAssignment/StopWords')

df=pd.read_excel('/content/20211030TestAssignment/Input.xlsx')
url_mapping= dict(zip(df['URL_ID'],df['URL']))


def analyze_text(file_path):
  with open(file_path,'r',encoding='utf-8') as file:
    text = file.read()

  file_name = os.path.basename(file_path)
  URL_ID = file_name.split('.')[0]
  URL = url_mapping.get(URL_ID,None)

  words = word_tokenize(text.lower())
  cleaned_words = [word for word in words if word.isalpha() and word not in stop_words]

  positive_score = sum(1 for word in cleaned_words if word in positive_words)
  negative_score = sum(-1 for word in cleaned_words if word in negative_words)
  negative_score = -1 * negative_score

  polarity_score = (positive_score - negative_score)/((positive_score + negative_score) + 0.000001)
  subjectivity_score = (positive_score + negative_score)/(len(cleaned_words) + 0.000001)

  sentences = sent_tokenize(text)
  num_sentences = len(sentences)

  num_words = len(cleaned_words)
  avg_words_per_sentence = num_words / num_sentences if num_sentences > 0 else 0

  avg_sentence_length = num_words / num_sentences if num_sentences > 0 else 0

  complex_words = [word for word in cleaned_words if syllable_count(word)>=2]
  percent_complex_words = (len(complex_words)/num_words) * 100

  fog_index = 0.4 * (avg_sentence_length + percent_complex_words)

  syllables_per_word = sum(syllable_count(word) for word in cleaned_words) / num_words if num_words > 0 else 0
  avg_word_length = sum(len(word) for word in cleaned_words) / num_words if num_words > 0 else 0
  personal_pronouns = len(re.findall(r'\b(I|we|my|ours|us)\b', text, flags=re.I))

  return {
        'URL_ID' : URL_ID,
        'URL' : URL,
        'POSITIVE SCORE': positive_score,
        'NEGATIVE SCORE': negative_score,
        'POLARITY SCORE': polarity_score,
        'SUBJECTIVITY SCORE': subjectivity_score,
        'AVG SENTENCE LENGTH': avg_sentence_length,
        'PERCENTAGE OF COMPLEX WORD': percent_complex_words,
        'FOG INDEX': fog_index,
        'AVG NUMBER OF WORDS PER SENTENCE':avg_words_per_sentence,
        'COMPLEX WORD COUNT': len(complex_words),
        'WORD COUNT': num_words,
        'SYLLABLE PER WORD': syllables_per_word,
        'PERSONAL PRONOUNS': personal_pronouns,
        'AVG WORD LENGTH': avg_word_length
    }


In [ ]:
def analyze_articles(input_excel_sheet,input_dir, output_file):
    df = pd.read_excel(input_excel_sheet)
    data = []
    for index, row in df.iterrows():
        file_path = os.path.join(input_dir,f"{row['URL_ID']}.txt")
        try:
            result = analyze_text(file_path)
            result['URL_ID'] = row['URL_ID']
            result['URL']    = row['URL']
            data.append(result)
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
            continue

    if data:
        df_output = pd.DataFrame(data)
        df_output.to_excel(output_file, index=False)
    else:
        print("No data to write to Excel.")


In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
if __name__ == "__main__":
  input_file = '/content/20211030TestAssignment/Input.xlsx'
  output_dir = '/content/20211030TestAssignment/extracted_text_files'
  output_file = '/content/20211030TestAssignment/OutputDataStructure.xlsx'
  input_excel_file = '/content/20211030TestAssignment/OutputDataStructure.xlsx'
  print("EXTRACTING ARTICLES....")
  extract_articles(input_file,output_dir)

  print("Analyzing articles...")
  analyze_articles(output_file,output_dir, output_file)

  print("Processing complete. Results saved in Output_Data_Structure.xlsx.")

EXTRACTING ARTICLES....
Analyzing articles...
Processing complete. Results saved in Output_Data_Structure.xlsx.
